In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.types import IntegerType, LongType, DecimalType,StructType, StructField, StringType
from pyspark.sql import Row
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.sql import Window

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
ssc = StreamingContext(sc, 5)

stream_data = ssc.textFileStream("/test/")


def readMyStream(rdd):
    if not rdd.isEmpty():
        df = spark.read.option("delimiter", "\t").csv(rdd)
        print('Started the Process')
        print('Selection of Columns')
        df = df.filter(df._c1 > 0.5)
        #df = df.select('t1','t2','t3','timestamp').where(col("timestamp").isNotNull())
        df.show()
        df.coalesce(1).write.option("delimiter", "\t").text("/newdatacsv")

stream_data.foreachRDD( lambda rdd: readMyStream(rdd) )
ssc.start()
#ssc.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/29 20:00:23 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/29 20:00:23 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/29 20:00:23 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/29 20:00:23 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
22/04/29 20:00:31 WARN org.apache.spark.streaming.StreamingContext: Dynamic Allocation is enabled for this application. Enabling Dynamic allocation for Spark Streaming applications can cause data loss if Write Ahead Log is not enabled for non-replayable sources. See the programming guide for details on how to enable the Write Ahead Log.


In [2]:
ssc.stop()